In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

from time import time
from icecream import ic

from msfm.utils import analysis, parameters
from msfm import fiducial_pipeline

### constants

In [3]:
conf = analysis.load_config()

# tfr_pattern = "/cluster/scratch/athomsen/CosmoGrid/v1/fiducial/DESy3_fiducial_???.tfrecord"
tfr_pattern = "/Users/arne/data/DESY3/tfrecords/v2/DESy3_fiducial_000.tfrecord"
tfr_pattern = "/pscratch/sd/a/athomsen/DESY3/v2/fiducial/DESy3_fiducial_???.tfrecord"

params = ["Om", "s8", "Aia"]
pert_labels = parameters.get_fiducial_perturbation_labels(params)
n_perts = len(pert_labels)

n_z_bins = 4
batch_size = 4
examples_shuffle_buffer = 10
n_readers = 1
n_prefetch = 3
n_batches = 10

23-02-28 15:28:13  analysis.py INF   Loaded the config 


### strategy

In [4]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.get_strategy()

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2023-02-28 15:28:13.186621: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-28 15:28:13.186758: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
def dataset_fn(input_context):
    dset = fiducial_pipeline.get_fiducial_dset(
        tfr_pattern,
        pert_labels,
        batch_size,
        conf=None,
        n_batches=n_batches,
        i_noise=0,
        n_readers=n_readers,
        n_prefetch=n_prefetch,
        examples_shuffle_buffer=examples_shuffle_buffer,
        input_context=input_context
    )

    return dset

dist_dset = strategy.distribute_datasets_from_function(dataset_fn)

for data_vectors, index in dist_dset:
    print(data_vectors.shape)

23-02-28 15:28:13 fiducial_pip INF   Starting to generate the fiducial training set for i_noise = 0 
23-02-28 15:28:13  analysis.py INF   Loaded the config 
23-02-28 15:28:13  analysis.py INF   Loaded the pixel file 
23-02-28 15:28:13  analysis.py INF   Loaded the config 
23-02-28 15:28:13  analysis.py INF   Loaded the pixel file 
23-02-28 15:28:13 fiducial_pip INF   Sharding the dataset over 1 replicas 
23-02-28 15:28:13 fiducial_pip INF   Using a local batch size of 4 per replica 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fi

2023-02-28 15:28:14.857801: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-28 15:28:14.883261: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-28 15:28:14.887364: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)


In [6]:
def dataset_fn(input_context):
    dset = fiducial_pipeline.get_fiducial_multi_noise_dset(
        tfr_pattern,
        pert_labels,
        batch_size,
        conf=None,
        n_batches=n_batches,
        n_noise=3,
        n_readers=n_readers,
        n_prefetch=n_prefetch,
        examples_shuffle_buffer=examples_shuffle_buffer,
        input_context=input_context
    )

    return dset

dist_dset = strategy.distribute_datasets_from_function(dataset_fn)

for data_vectors, index in dist_dset:
    print(data_vectors.shape)

23-02-28 15:28:22 fiducial_pip INF   Starting to generate the fiducial training set for i_noise = 0 
23-02-28 15:28:22  analysis.py INF   Loaded the config 
23-02-28 15:28:22  analysis.py INF   Loaded the pixel file 
23-02-28 15:28:22  analysis.py INF   Loaded the config 
23-02-28 15:28:22  analysis.py INF   Loaded the pixel file 
23-02-28 15:28:22 tfrecords.py WAR   Tracing parse_inverse_fiducial 
23-02-28 15:28:22  analysis.py INF   Loaded the config 
23-02-28 15:28:22 fiducial_pip WAR   Tracing dset_add_bias 
23-02-28 15:28:22 fiducial_pip WAR   Tracing dset_add_noise 
23-02-28 15:28:22 fiducial_pip WAR   Tracing dset_concat_perts 
23-02-28 15:28:22 fiducial_pip INF   Successfully generated the fiducial training set with element_spec (TensorSpec(shape=(28, 463872, 4), dtype=tf.float32, name=None), (TensorSpec(shape=(4,), dtype=tf.int64, name=None), TensorSpec(shape=(4,), dtype=tf.int32, name=None))) for i_noise = 0 
23-02-28 15:28:22 fiducial_pip INF   Starting to generate the fiduc

2023-02-28 15:28:23.608679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-28 15:28:23.612711: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
(28, 463872, 4)
